## Quantitative value investing (Systematic value investing)
- Form of value investing that analyses fundamnetal data such as financial statement line items, economic data, unstructured data
- Portfolio diversification and rebalancing method 
- Ranking the market and setting buy/sell rules
- This project will build an investing strategy that selects the 50 stocks with the best value metrics
- This will determine and calculate the recommended trades for an equal-weight portfolio of these 50 stocks

In [ ]:
import numpy as np 
import pandas as pd 
import xlsxwriter 
import requests
from scipy import stats
import math 

In [3]:
stocks = pd.read_csv('starter/sp_500_stocks.csv')
from starter.secrets import IEX_CLOUD_API_TOKEN

In [ ]:
"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/{field}"